In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz

     |████████████████████████████████| 827.9 MB 4.1 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=e2bbe6ba88bccdb9dbcab5a65207d8b3328bc6ae67d83771c43e6d544629d1d3
  Stored in directory: /root/.cache/pip/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.7 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_US		   heatmap529_UK.ipynb
 a0529.csv		   heatmap530.ipynb
 a0530.csv		   Heat_Map_Online_language.ipynb
 csv			   __MACOSX
 csv.zip		   non_violant1.csv
 dataframe_csv		   non_violant2.csv
 dataframe_csv_UK	   UK
 dataframe_result_csv	   violant.csv
 dataframe_result_csv_UK  '新建 Microsoft PowerPoint 演示文稿.pptx'
 Github


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_531 = 'UK/05_30.csv'

In [8]:
ct = pd.read_csv(ct_path_531)

In [9]:
ct.shape[1]

47

In [10]:
#columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [22]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg
0,575867902590661_1494105610766881,Facebook,2020-05-30 23:59:56,2021-07-19 07:35:46,photo,,NaN,Tomorrow's Sunday National front page: Global ...,OPEN JOINT LETTER: The Lord Advocate Mr Walter...,1.0,https://www.facebook.com/thenationalnewspapers...,https://www.facebook.com/575867902590661/posts...,0,-2.951220,1.0,28,8,1,4,0,0,0,0,0,0,55,12,6,11,2,23,2,10,0,2,12754183,Aye Aberdeen,AyeAberdeen,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,"Since Brexit, Scotland has been repeatedly ign...",False,2016-07-07 17:40:55,en,0,12754183|1494105610766881,"{'Edinburgh': 1, 'Scotland': 6, 'UK': 1, 'US':..."
1,174766919255146_271144980752352,Facebook,2020-05-30 23:59:52,2021-07-01 21:21:40,live_video_complete,,NaN,Tomorrow's Sunday National front page: Global ...,George Floyd Memorial Go Fund Me: https://www....,4.0,https://www.facebook.com/SweetH20MusicHall/vid...,https://www.facebook.com/174766919255146/posts...,48081,1.500000,2.0,33,13,20,11,0,0,0,1,0,1,11,9,19,9,1,1,1,1,0,2,403510,Sweetwater Music Hall,SweetH20MusicHall,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Sweetwater Music Hall is a landmark live music...,True,2011-07-10 22:02:00,en,0,403510|271144980752352,{}
2,104129837848_10157318590807849,Facebook,2020-05-30 23:59:50,2021-08-31 10:53:12,link,Cities brace for increasing unrest as anger ov...,kgun9.com,"Protesters set police cars ablaze, smashed bus...","Protesters set police cars ablaze, smashed bus...",1.0,https://www.kgun9.com/news/national/cities-bra...,https://www.facebook.com/104129837848/posts/10...,200968,-1.194444,1.0,19,3,80,0,1,2,13,26,0,1,54,14,45,13,6,12,10,18,0,3,20324,KGUN 9 On Your Side,KGUN9OnYourSide,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Visit us at kgun9.com\nFollow us on IG https:/...,True,2009-06-11 22:36:58,en,0,20324|10157318590807849,"{'Atlanta': 1, 'Los Angeles': 1}"
3,346979175820702_883214425530505,Facebook,2020-05-30 23:59:50,2021-06-18 19:51:53,link,Hundreds of thousands stage multiracial demons...,wsws.org,Mass demonstrations of workers and youth of al...,The police are not a cross section of American...,1.0,https://www.wsws.org/en/articles/2020/05/30/pe...,https://www.facebook.com/346979175820702/posts...,0,-0.750000,1.0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,14859738,Victoria University of Wellington Students for...,vuwiysse,https://scontent-den4-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Associated with IYSSE and the Social Equality ...,False,2018-03-21 22:42:39,en,0,14859738|883214425530505,{}
4,624208120942702_3471299859566833,Facebook,2020-05-30 23:59:50,2021-05-27 00:13:54,link,Anglican Church in North America,anglicanchurch.net,Anglican Church in North America,We are grateful for this letter from ACNA bish...,2.0,http://www.anglicanchurch.net/?%2Fmain%2Fpage%...,https://www.facebook.com/624208120942702/posts...,804,0.684211,1.0,11,0,0,2,0,0,0,0,0,0,4,1,1,2,0,0,0,0,0,0,3518248,Immanuel Anglican Church of Chicago,chicagoanglican,https://scontent-den4-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Immanuel Anglican Church is a congregation in ...,False,2013-05-14 02:31:46,en,0,3518248|3471299859566833,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [23]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [24]:
ct['NER-msg'] = ct['message'].apply(ner)

In [25]:
ct['NER-desc'] = ct['description'].apply(ner)

In [26]:
ct['NER-title'] = ct['title'].apply(ner)

In [27]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Edinburgh': 1, 'Scotland': 6, 'UK': 1, 'US':...",{}
1,{},{}
2,"{'Atlanta': 1, 'Los Angeles': 1}","{'Atlanta': 1, 'Los Angeles': 1}"
3,{},{}
4,{},{}


In [28]:
ct.to_csv('a0530.csv')

In [29]:
ct=pd.read_csv('a0530.csv')

In [30]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [31]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [32]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Edinburgh': 1, 'Scotland': 6, 'UK': 1, 'US':...",{},{}
2,"{'Atlanta': 1, 'Los Angeles': 1}","{'Atlanta': 1, 'Los Angeles': 1}",{}
5,{},{'Minneapolis': 1},{'US': 1}
6,{},{},{'US': 1}
9,{'Utica': 1},"{'Utica': 1, 'NY': 1, 'Kem': 1}",{}


In [33]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [34]:
ct_ner['NER-msg']

0        [Iraq, UK, Edinburgh, Scotland, US, Convenor]
2                               [Los Angeles, Atlanta]
5                                                   []
6                                                   []
9                                              [Utica]
                             ...                      
35362                                        [Atlanta]
35363                                        [Atlanta]
35364                     [NYC, Minneapolis, Brooklyn]
35365                             [Dallas, Deep Ellum]
35366                                             [us]
Name: NER-msg, Length: 27746, dtype: object

In [35]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
ct_ner['NER']

0        [Iraq, UK, Edinburgh, Scotland, US, Convenor]
2         [Los Angeles, Atlanta, Los Angeles, Atlanta]
5                                    [Minneapolis, US]
6                                                 [US]
9                              [Utica, Utica, NY, Kem]
                             ...                      
35362                               [Atlanta, Atlanta]
35363                               [Atlanta, Atlanta]
35364            [NYC, Minneapolis, Brooklyn, Atlanta]
35365                    [Dallas, Deep Ellum, Atlanta]
35366                                             [us]
Name: NER, Length: 27746, dtype: object

In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
ner_lst = ct_ner['NER'].values

In [40]:
ner_lst

array([list(['iraq', 'uk', 'edinburgh', 'scotland', 'us', 'convenor']),
       list(['los angeles', 'atlanta']), list(['minneapolis', 'us']), ...,
       list(['nyc', 'atlanta', 'minneapolis', 'brooklyn']),
       list(['deep ellum', 'dallas', 'atlanta']), list(['us'])],
      dtype=object)

In [41]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [42]:
list

['iraq',
 'uk',
 'edinburgh',
 'scotland',
 'us',
 'convenor',
 'los angeles',
 'atlanta',
 'minneapolis',
 'us',
 'us',
 'utica',
 'ny',
 'kem',
 'rutland',
 'burlington',
 'minneapolis',
 'u.s.',
 'vermont',
 'brattleboro',
 'us',
 'montpelier',
 'the united states',
 'minneapolis',
 'america',
 'hennepin county',
 'minneapolis',
 'minnesota',
 'www.howaboutthat.biza',
 'minneapolis',
 'minneapolis',
 'd.c.',
 'minneapolis',
 'nyc',
 'new york city',
 'minneapolis',
 'brooklyn',
 'nyc',
 'richfield',
 'brooklyn',
 'nyc',
 'richfield',
 'south bend',
 'brooklyn',
 'richfield',
 'minneapolis',
 'the united states',
 'united states',
 'united states',
 'richfield',
 'minneapolis',
 '🇸',
 'minneapolis',
 'atlanta',
 'michigan',
 'atlanta',
 'minneapolis',
 'flint twp',
 'minnesota',
 'minneapolis',
 'seattle',
 'minneapolis',
 'seattle',
 'america',
 'seattle',
 'lake worth beach city',
 'minneapolis',
 'los angeles',
 'us',
 'minneapolis',
 'rutland',
 'burlington',
 'minneapolis',
 'u.

In [43]:
c = collections.Counter()

In [44]:
dict_a=collections.Counter(list).most_common()

In [45]:
dict_a

[('minneapolis', 15748),
 ('minnesota', 4208),
 ('america', 2950),
 ('us', 2725),
 ('atlanta', 2491),
 ('u.s.', 1732),
 ('the united states', 1178),
 ('houston', 760),
 ('new york', 735),
 ('washington', 711),
 ('chicago', 611),
 ('los angeles', 549),
 ('oakland', 494),
 ('detroit', 485),
 ('georgia', 454),
 ('louisville', 429),
 ('brooklyn', 425),
 ('new york city', 423),
 ('california', 397),
 ('boston', 396),
 ('portland', 386),
 ('denver', 377),
 ('d.c.', 363),
 ('dallas', 361),
 ('hennepin county', 343),
 ('texas', 341),
 ('philadelphia', 333),
 ('st. paul', 317),
 ('usa', 316),
 ('milwaukee', 310),
 ('dc', 292),
 ('san jose', 286),
 ('mn', 271),
 ('nyc', 270),
 ('columbus', 269),
 ('seattle', 220),
 ('ohio', 215),
 ('newark', 212),
 ('tennessee', 199),
 ('charlotte', 189),
 ('phoenix', 184),
 ('mississippi', 183),
 ('florida', 182),
 ('kentucky', 178),
 ('the united states of america', 173),
 ('michigan', 169),
 ('cincinnati', 165),
 ('indianapolis', 162),
 ('nashville', 160),
 (

In [46]:
dataframe531=pd.DataFrame(dict_a)

In [47]:
len(dict_a)

2945

In [48]:
dataframe531.to_csv('dataframe0530_UK.csv')